In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.layers import Input
from keras.models import Model
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
import pickle
from sklearn.metrics import log_loss

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    elif epoch<50:
        return 0.0001
    else:
        return 0.00005

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten
from keras import layers
from keras.models import Model

def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):

    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

def create_incept_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # bn
    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
    x = conv2d_bn(x, 32, 3, 3, padding='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 64, 1, 1, padding='valid')
    x = conv2d_bn(x, 64, 3, 3, padding='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # mixed 0, 1, 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')
    
    # mixed 1: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 96, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 96, 5, 5)

    branch3x3dbl = conv2d_bn(x, 96, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 96, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')
    
    # mixed 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 96, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 96, 5, 5)

    branch3x3dbl = conv2d_bn(x, 96, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 96, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')

    x = Conv2D(96, 3, strides=2, padding='same',activation='relu')(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')

model model


In [5]:
def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            rotation_range = 20,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.2,
            horizontal_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_incept_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.58805, saving model to best_m.h5
 - 9s - loss: 0.5672 - acc: 0.7187 - val_loss: 0.5880 - val_acc: 0.6384
Epoch 2/100
Epoch 00002: val_loss improved from 0.58805 to 0.57635, saving model to best_m.h5
 - 4s - loss: 0.4424 - acc: 0.7881 - val_loss: 0.5764 - val_acc: 0.6309
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 4s - loss: 0.4378 - acc: 0.8039 - val_loss: 0.7828 - val_acc: 0.7631
Epoch 4/100
Epoch 00004: val_loss improved from 0.57635 to 0.55974, saving model to best_m.h5
 - 4s - loss: 0.3936 - acc: 0.8092 - val_loss: 0.5597 - val_acc: 0.6983
Epoch 5/100
Epoch 00005: val_loss improved from 0.55974 to 0.36779, saving model to best_m.h5
 - 4s - loss: 0.3998 - acc: 0.8218 - val_loss: 0.3678 - val_acc: 0.8354
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 4s - loss: 0.3402 - acc: 0.8500 - val_loss: 0.6321 - val_acc: 0.8279
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 4s - loss: 0.3622 - acc: 0.8372 - val_loss:

Epoch 65/100
Epoch 00065: val_loss did not improve
 - 3s - loss: 0.1430 - acc: 0.9431 - val_loss: 0.3275 - val_acc: 0.8878
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 3s - loss: 0.1839 - acc: 0.9301 - val_loss: 0.3358 - val_acc: 0.8603
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 3s - loss: 0.1440 - acc: 0.9348 - val_loss: 0.3235 - val_acc: 0.8803
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 3s - loss: 0.1256 - acc: 0.9517 - val_loss: 0.3395 - val_acc: 0.8753
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 3s - loss: 0.1750 - acc: 0.9250 - val_loss: 0.3355 - val_acc: 0.8703
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 3s - loss: 0.1202 - acc: 0.9567 - val_loss: 0.3363 - val_acc: 0.8703
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 3s - loss: 0.1371 - acc: 0.9383 - val_loss: 0.3389 - val_acc: 0.8603
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 3s - loss: 0.1456 - acc: 0.9323 - val_loss: 0.3477 - val_acc: 0.8753
Epoch 73/100
Epo

Epoch 30/100
Epoch 00030: val_loss did not improve
 - 3s - loss: 0.2693 - acc: 0.8831 - val_loss: 0.2021 - val_acc: 0.9077
Epoch 31/100
Epoch 00031: val_loss did not improve
 - 3s - loss: 0.2079 - acc: 0.9158 - val_loss: 0.2241 - val_acc: 0.9027
Epoch 32/100
Epoch 00032: val_loss improved from 0.20033 to 0.19750, saving model to best_m.h5
 - 4s - loss: 0.2492 - acc: 0.9045 - val_loss: 0.1975 - val_acc: 0.9077
Epoch 33/100
Epoch 00033: val_loss improved from 0.19750 to 0.19573, saving model to best_m.h5
 - 4s - loss: 0.1922 - acc: 0.9233 - val_loss: 0.1957 - val_acc: 0.9102
Epoch 34/100
Epoch 00034: val_loss did not improve
 - 3s - loss: 0.2134 - acc: 0.9164 - val_loss: 0.2103 - val_acc: 0.9077
Epoch 35/100
Epoch 00035: val_loss improved from 0.19573 to 0.19083, saving model to best_m.h5
 - 4s - loss: 0.2050 - acc: 0.9150 - val_loss: 0.1908 - val_acc: 0.9102
Epoch 36/100
Epoch 00036: val_loss did not improve
 - 3s - loss: 0.2451 - acc: 0.9018 - val_loss: 0.2108 - val_acc: 0.9152
Epoch 3

Epoch 96/100
Epoch 00096: val_loss did not improve
 - 3s - loss: 0.1188 - acc: 0.9573 - val_loss: 0.2506 - val_acc: 0.9052
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 3s - loss: 0.1017 - acc: 0.9533 - val_loss: 0.2537 - val_acc: 0.9102
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 4s - loss: 0.1230 - acc: 0.9508 - val_loss: 0.3325 - val_acc: 0.8978
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 3s - loss: 0.1126 - acc: 0.9539 - val_loss: 0.2633 - val_acc: 0.9027
Epoch 100/100
Epoch 00100: val_loss did not improve
 - 4s - loss: 0.1097 - acc: 0.9558 - val_loss: 0.2386 - val_acc: 0.9102
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.69473, saving model to best_m.h5
 - 14s - loss: 0.5965 - acc: 0.7178 - val_loss: 0.6947 - val_acc: 0.5087
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 4s - loss: 0.4038 - acc: 0.8042 - val_loss: 2.2046 - val_acc: 0.5087
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 4s - loss: 0.4375 - acc: 0.8053 - val_loss: 1.

Epoch 61/100
Epoch 00061: val_loss did not improve
 - 4s - loss: 0.1579 - acc: 0.9373 - val_loss: 0.2574 - val_acc: 0.9077
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 4s - loss: 0.1565 - acc: 0.9342 - val_loss: 0.2461 - val_acc: 0.9152
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 4s - loss: 0.1544 - acc: 0.9400 - val_loss: 0.2468 - val_acc: 0.9177
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 3s - loss: 0.1407 - acc: 0.9467 - val_loss: 0.2661 - val_acc: 0.9077
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 3s - loss: 0.1283 - acc: 0.9439 - val_loss: 0.2580 - val_acc: 0.9152
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 3s - loss: 0.1369 - acc: 0.9383 - val_loss: 0.2549 - val_acc: 0.9127
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 4s - loss: 0.1427 - acc: 0.9481 - val_loss: 0.3321 - val_acc: 0.8828
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 3s - loss: 0.1146 - acc: 0.9558 - val_loss: 0.2519 - val_acc: 0.9227
Epoch 69/100
Epo

Epoch 25/100
Epoch 00025: val_loss did not improve
 - 3s - loss: 0.2579 - acc: 0.8875 - val_loss: 0.2447 - val_acc: 0.8978
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 3s - loss: 0.2528 - acc: 0.9000 - val_loss: 0.2373 - val_acc: 0.8953
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 3s - loss: 0.2548 - acc: 0.9092 - val_loss: 0.2337 - val_acc: 0.8928
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 3s - loss: 0.3108 - acc: 0.8809 - val_loss: 0.2707 - val_acc: 0.8828
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 3s - loss: 0.2504 - acc: 0.8983 - val_loss: 0.3485 - val_acc: 0.8354
Epoch 30/100
Epoch 00030: val_loss improved from 0.21611 to 0.19309, saving model to best_m.h5
 - 4s - loss: 0.2343 - acc: 0.9000 - val_loss: 0.1931 - val_acc: 0.9202
Epoch 31/100
Epoch 00031: val_loss improved from 0.19309 to 0.19209, saving model to best_m.h5
 - 4s - loss: 0.2097 - acc: 0.9108 - val_loss: 0.1921 - val_acc: 0.9202
Epoch 32/100
Epoch 00032: val_loss did not improve


Epoch 91/100
Epoch 00091: val_loss did not improve
 - 4s - loss: 0.1259 - acc: 0.9523 - val_loss: 0.3258 - val_acc: 0.8753
Epoch 92/100
Epoch 00092: val_loss did not improve
 - 3s - loss: 0.1240 - acc: 0.9483 - val_loss: 0.2800 - val_acc: 0.9127
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 3s - loss: 0.1312 - acc: 0.9517 - val_loss: 0.2852 - val_acc: 0.8928
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 3s - loss: 0.1216 - acc: 0.9450 - val_loss: 0.2722 - val_acc: 0.8953
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 3s - loss: 0.1213 - acc: 0.9500 - val_loss: 0.2634 - val_acc: 0.9027
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 3s - loss: 0.1385 - acc: 0.9464 - val_loss: 0.2650 - val_acc: 0.8953
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 3s - loss: 0.1227 - acc: 0.9558 - val_loss: 0.2620 - val_acc: 0.9052
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 3s - loss: 0.1240 - acc: 0.9550 - val_loss: 0.2547 - val_acc: 0.9027
Epoch 99/100
Epo

In [6]:
with open('../features/incept_aug2_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/incept_aug2_sub.csv', index=False)

# pre 2110
# new 2048

0.223411707071
         id  is_iceberg
0  5941774d    0.036655
1  4023181e    0.675880
2  b20200e4    0.167083
3  e7f018bb    0.992381
4  4371c8c3    0.030700


In [7]:
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters

    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    bn_axis = 3
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 128], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 128], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 128], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 256], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 256], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 256], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 256], stage=3, block='d')
    
    x = Conv2D(128, 3, strides=2, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [8]:
def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            rotation_range = 20,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.2,
            horizontal_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_resnet_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.74378, saving model to best_m.h5
 - 22s - loss: 1.0962 - acc: 0.6389 - val_loss: 0.7438 - val_acc: 0.4663
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 5s - loss: 0.4947 - acc: 0.7728 - val_loss: 1.1046 - val_acc: 0.4688
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 5s - loss: 0.4307 - acc: 0.8133 - val_loss: 1.2017 - val_acc: 0.6060
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 5s - loss: 0.4416 - acc: 0.8050 - val_loss: 0.8885 - val_acc: 0.6783
Epoch 5/100
Epoch 00005: val_loss improved from 0.74378 to 0.57043, saving model to best_m.h5
 - 5s - loss: 0.4782 - acc: 0.7922 - val_loss: 0.5704 - val_acc: 0.7955
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 5s - loss: 0.4187 - acc: 0.8172 - val_loss: 0.9705 - val_acc: 0.7107
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 5s - loss: 0.3748 - acc: 0.8300 - val_loss: 0.7404 - val_acc: 0.7955
Epoch 8/100
Epoch 00008: val_loss did not improve
 - 5s - los

Epoch 62/100
Epoch 00062: val_loss did not improve
 - 5s - loss: 0.1859 - acc: 0.9242 - val_loss: 0.2889 - val_acc: 0.8853
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 5s - loss: 0.2491 - acc: 0.9045 - val_loss: 0.2916 - val_acc: 0.8753
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 5s - loss: 0.1905 - acc: 0.9167 - val_loss: 0.2862 - val_acc: 0.8803
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 5s - loss: 0.2512 - acc: 0.9062 - val_loss: 0.2841 - val_acc: 0.8753
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 5s - loss: 0.1981 - acc: 0.9250 - val_loss: 0.2904 - val_acc: 0.8703
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 5s - loss: 0.2020 - acc: 0.9073 - val_loss: 0.3118 - val_acc: 0.8554
Epoch 68/100
Epoch 00068: val_loss improved from 0.28206 to 0.28038, saving model to best_m.h5
 - 5s - loss: 0.1840 - acc: 0.9250 - val_loss: 0.2804 - val_acc: 0.8703
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 5s - loss: 0.1988 - acc: 0.9225 - val_los

Epoch 25/100
Epoch 00025: val_loss did not improve
 - 5s - loss: 0.3441 - acc: 0.8522 - val_loss: 0.2830 - val_acc: 0.8554
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 5s - loss: 0.3177 - acc: 0.8583 - val_loss: 0.4453 - val_acc: 0.7406
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 5s - loss: 0.3034 - acc: 0.8733 - val_loss: 0.3024 - val_acc: 0.8778
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 5s - loss: 0.3291 - acc: 0.8531 - val_loss: 0.6258 - val_acc: 0.7007
Epoch 29/100
Epoch 00029: val_loss improved from 0.24357 to 0.23655, saving model to best_m.h5
 - 5s - loss: 0.3246 - acc: 0.8583 - val_loss: 0.2366 - val_acc: 0.8928
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 5s - loss: 0.3435 - acc: 0.8545 - val_loss: 0.3308 - val_acc: 0.8778
Epoch 31/100
Epoch 00031: val_loss improved from 0.23655 to 0.22096, saving model to best_m.h5
 - 5s - loss: 0.2565 - acc: 0.9056 - val_loss: 0.2210 - val_acc: 0.9127
Epoch 32/100
Epoch 00032: val_loss improved from 0.

Epoch 90/100
Epoch 00090: val_loss did not improve
 - 5s - loss: 0.2035 - acc: 0.9183 - val_loss: 0.2136 - val_acc: 0.9177
Epoch 91/100
Epoch 00091: val_loss did not improve
 - 5s - loss: 0.2084 - acc: 0.9131 - val_loss: 0.2285 - val_acc: 0.9002
Epoch 92/100
Epoch 00092: val_loss did not improve
 - 5s - loss: 0.2336 - acc: 0.9098 - val_loss: 0.2193 - val_acc: 0.9152
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 5s - loss: 0.2021 - acc: 0.9150 - val_loss: 0.2122 - val_acc: 0.9227
Epoch 94/100
Epoch 00094: val_loss improved from 0.19797 to 0.19493, saving model to best_m.h5
 - 5s - loss: 0.1913 - acc: 0.9170 - val_loss: 0.1949 - val_acc: 0.9252
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 5s - loss: 0.2013 - acc: 0.9183 - val_loss: 0.2102 - val_acc: 0.9252
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 5s - loss: 0.1831 - acc: 0.9258 - val_loss: 0.2020 - val_acc: 0.9202
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 5s - loss: 0.1730 - acc: 0.9348 - val_los

Epoch 52/100
Epoch 00052: val_loss improved from 0.24331 to 0.24248, saving model to best_m.h5
 - 5s - loss: 0.2468 - acc: 0.8967 - val_loss: 0.2425 - val_acc: 0.9227
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 5s - loss: 0.2120 - acc: 0.9183 - val_loss: 0.2527 - val_acc: 0.8878
Epoch 54/100
Epoch 00054: val_loss improved from 0.24248 to 0.23738, saving model to best_m.h5
 - 5s - loss: 0.2065 - acc: 0.9100 - val_loss: 0.2374 - val_acc: 0.9002
Epoch 55/100
Epoch 00055: val_loss did not improve
 - 6s - loss: 0.2376 - acc: 0.9012 - val_loss: 0.2486 - val_acc: 0.9102
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 5s - loss: 0.2280 - acc: 0.9025 - val_loss: 0.2551 - val_acc: 0.8953
Epoch 57/100
Epoch 00057: val_loss did not improve
 - 5s - loss: 0.2290 - acc: 0.9058 - val_loss: 0.2993 - val_acc: 0.8529
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 5s - loss: 0.2309 - acc: 0.9000 - val_loss: 0.2604 - val_acc: 0.9077
Epoch 59/100
Epoch 00059: val_loss did not improve


Epoch 15/100
Epoch 00015: val_loss did not improve
 - 6s - loss: 0.4209 - acc: 0.8149 - val_loss: 0.3237 - val_acc: 0.8803
Epoch 16/100
Epoch 00016: val_loss did not improve
 - 5s - loss: 0.4040 - acc: 0.8383 - val_loss: 0.3195 - val_acc: 0.8678
Epoch 17/100
Epoch 00017: val_loss did not improve
 - 5s - loss: 0.4024 - acc: 0.8241 - val_loss: 0.3795 - val_acc: 0.8055
Epoch 18/100
Epoch 00018: val_loss improved from 0.31611 to 0.30229, saving model to best_m.h5
 - 6s - loss: 0.4070 - acc: 0.8306 - val_loss: 0.3023 - val_acc: 0.8903
Epoch 19/100
Epoch 00019: val_loss did not improve
 - 5s - loss: 0.4173 - acc: 0.8203 - val_loss: 0.3237 - val_acc: 0.8803
Epoch 20/100
Epoch 00020: val_loss improved from 0.30229 to 0.28992, saving model to best_m.h5
 - 5s - loss: 0.3764 - acc: 0.8239 - val_loss: 0.2899 - val_acc: 0.8853
Epoch 21/100
Epoch 00021: val_loss did not improve
 - 5s - loss: 0.3962 - acc: 0.8220 - val_loss: 0.3024 - val_acc: 0.8778
Epoch 22/100
Epoch 00022: val_loss improved from 0.

Epoch 78/100
Epoch 00078: val_loss did not improve
 - 5s - loss: 0.2455 - acc: 0.9031 - val_loss: 0.2226 - val_acc: 0.9252
Epoch 79/100
Epoch 00079: val_loss did not improve
 - 5s - loss: 0.2237 - acc: 0.9108 - val_loss: 0.2279 - val_acc: 0.9152
Epoch 80/100
Epoch 00080: val_loss did not improve
 - 5s - loss: 0.2490 - acc: 0.9050 - val_loss: 0.2173 - val_acc: 0.9252
Epoch 81/100
Epoch 00081: val_loss improved from 0.20131 to 0.19633, saving model to best_m.h5
 - 5s - loss: 0.2315 - acc: 0.9175 - val_loss: 0.1963 - val_acc: 0.9327
Epoch 82/100
Epoch 00082: val_loss did not improve
 - 5s - loss: 0.2326 - acc: 0.9142 - val_loss: 0.2112 - val_acc: 0.9252
Epoch 83/100
Epoch 00083: val_loss did not improve
 - 5s - loss: 0.2185 - acc: 0.9192 - val_loss: 0.1990 - val_acc: 0.9377
Epoch 84/100
Epoch 00084: val_loss did not improve
 - 5s - loss: 0.2128 - acc: 0.9200 - val_loss: 0.2115 - val_acc: 0.9302
Epoch 85/100
Epoch 00085: val_loss did not improve
 - 5s - loss: 0.2500 - acc: 0.8998 - val_los

In [9]:
with open('../features/resnet_aug2_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/resnet_aug2_sub.csv', index=False)

# pre 2271
# new 2118

0.223741863762
         id  is_iceberg
0  5941774d    0.136624
1  4023181e    0.420358
2  b20200e4    0.003222
3  e7f018bb    0.960796
4  4371c8c3    0.111984
